In [1]:
import numpy as np

def matmul_blocked(A: np.ndarray, B: np.ndarray, BS: int = 64) -> np.ndarray:
    """
    C = A @ B with cache-friendly tiling.
    A: (M, K), B: (K, N) -> C: (M, N)
    BS: tile (block) size along each dimension
    """
    M, K = A.shape
    K2, N = B.shape
    assert K == K2

    # Use a reasonable dtype for accumulation (prevents precision loss if inputs are int/float16)
    C = np.zeros((M, N), dtype=np.result_type(A, B, np.float32))

    # --- Outer 2D tiling over C (i,j) tiles; inner tiling over k (the reduction dim) ---
    for ii in range(0, M, BS):          # iterate C’s rows in blocks [ii : ii+BS)
        i_end = min(ii + BS, M)         # handle edge tiles (when M is not multiple of BS)
        for jj in range(0, N, BS):      # iterate C’s cols in blocks [jj : jj+BS)
            j_end = min(jj + BS, N)

            # Create a view onto the C tile we’re currently producing.
            # Cij has shape (i_block, j_block) where i_block=i_end-ii, j_block=j_end-jj.
            Cij = C[ii:i_end, jj:j_end]

            # Sweep the reduction dimension in blocks (kk ... kk+BS).
            # We keep the C tile "hot" in cache while we accumulate contributions from each A/B k-tile.
            for kk in range(0, K, BS):
                k_end = min(kk + BS, K)

                # Aik is the tile of A that contributes to Cij for this k-block
                # shape: (i_block, k_block)
                Aik = A[ii:i_end, kk:k_end]

                # Bkj is the tile of B that contributes to Cij for this k-block
                # shape: (k_block, j_block)
                Bkj = B[kk:k_end, jj:j_end]

                # Micro-kernel: multiply the two small tiles and accumulate into Cij.
                # Using @ here calls BLAS on a small problem size (fast and vectorized).
                Cij += Aik @ Bkj

            # Optional: since Cij is a view into C, writing back is not strictly necessary.
            C[ii:i_end, jj:j_end] = Cij

    return C

In [2]:
np.random.seed(0)
A = np.random.randn(257, 513).astype(np.float32)
B = np.random.randn(513, 129).astype(np.float32)

C1 = A @ B
C2 = matmul_blocked(A, B, BS=64)
print("max abs diff:", np.max(np.abs(C1 - C2)))  # ~1e-5 to 1e-6

max abs diff: 4.386902e-05
